In [1]:
import numpy as np
import time
from math import sin, exp, pi

In [2]:
# Trapezoidal rule implementation in Python
def py_trapz(f, a, b, n):
    """
    Approximates the integral of f(x) from a to b using the trapezoidal rule.
    
    Parameters:
    f (function): The function to integrate.
    a (float): Start of the integration interval.
    b (float): End of the integration interval.
    n (int): Number of trapezoids.
    
    Returns:
    float: Approximation of the integral.
    """
    h = (b - a) / n
    result = 0.5 * (f(a) + f(b))
    for i in range(1, n):
        result += f(a + i * h)
    result *= h
    return result

In [3]:
%load_ext Cython

In [4]:
%%cython
import cython
import time
from libc.math cimport sin, exp

@cython.boundscheck(False)
@cython.wraparound(False)
def cy_trapz(f, double a, double b, int n):
    
    cdef double h = (b - a) / n
    cdef double result = 0.5 * (f(a) + f(b))
    cdef int i
    for i in range(1, n):
        result += f(a + i * h)
    result *= h
    return result


In [5]:
def numpy_trapz(f, a, b, n):
    x = np.linspace(a, b, n+1)
    y = f(x)
    return np.trapz(y, x)


In [6]:
def f1(x): return x**2
def f2(x): return np.sin(x)
def f3(x): return np.exp(x)
def f4(x): return 1 / x


In [7]:
def benchmark(f, a, b, n):

    print(f"Function: {f.__name__}, Interval: [{a}, {b}], Trapezoids: {n}")
    
    # Pure Python implementation
    start = time.time()
    result_python = py_trapz(f, a, b, n)
    end = time.time()
    print(f"Python Result: {result_python}, Time: {end - start:.6f}s")
    
    # Cython implementation
    start = time.time()
    result_cython = cy_trapz(f, a, b, n)  # Ensure cy_trapz is compiled
    end = time.time()
    print(f"Cython Result: {result_cython}, Time: {end - start:.6f}s")
    
    # NumPy implementation
    start = time.time()
    result_numpy = numpy_trapz(f, a, b, n)
    end = time.time()
    print(f"NumPy Result: {result_numpy}, Time: {end - start:.6f}s")
    print()


In [8]:
benchmark(f1, 0, 1, 1000)
benchmark(f2, 0, pi, 1000)
benchmark(f3, 0, 1, 1000)
benchmark(f4, 1, 2, 1000)


Function: f1, Interval: [0, 1], Trapezoids: 1000
Python Result: 0.33333349999999995, Time: 0.000168s
Cython Result: 0.33333349999999995, Time: 0.000132s
NumPy Result: 0.3333335, Time: 0.000366s

Function: f2, Interval: [0, 3.141592653589793], Trapezoids: 1000
Python Result: 1.9999983550656624, Time: 0.000770s
Cython Result: 1.9999983550656624, Time: 0.000681s
NumPy Result: 1.9999983550656628, Time: 0.000632s

Function: f3, Interval: [0, 1], Trapezoids: 1000
Python Result: 1.7182819716491962, Time: 0.000757s
Cython Result: 1.7182819716491962, Time: 0.000748s
NumPy Result: 1.718281971649195, Time: 0.000259s

Function: f4, Interval: [1, 2], Trapezoids: 1000
Python Result: 0.6931472430599374, Time: 0.000126s
Cython Result: 0.6931472430599374, Time: 0.000109s
NumPy Result: 0.6931472430599375, Time: 0.000320s



In [9]:
benchmark(f1, 0, 10, 10_000_000)


Function: f1, Interval: [0, 10], Trapezoids: 10000000
Python Result: 333.33333333334724, Time: 1.642045s
Cython Result: 333.33333333334724, Time: 1.221163s
NumPy Result: 333.3333333333349, Time: 0.162623s



In [10]:
import numpy as np
from scipy.integrate import quad

def f1(x): return x**2
def f2(x): return np.sin(x)
def f3(x): return np.exp(x)
def f4(x): return 1/x

intervals = {
    'f1': (0, 1),      
    'f2': (0, np.pi),  
    'f3': (0, 1),      
    'f4': (1, 2)      
}

results = {
    'f1': quad(f1, *intervals['f1']),
    'f2': quad(f2, *intervals['f2']),
    'f3': quad(f3, *intervals['f3']),
    'f4': quad(f4, *intervals['f4'])
}

for func, result in results.items():
    value, error = result
    print(f"Actual value of integral for {func}: {value:.6f} (Estimated error: {error:.6e})")


Actual value of integral for f1: 0.333333 (Estimated error: 3.700743e-15)
Actual value of integral for f2: 2.000000 (Estimated error: 2.220446e-14)
Actual value of integral for f3: 1.718282 (Estimated error: 1.907676e-14)
Actual value of integral for f4: 0.693147 (Estimated error: 7.695480e-15)
